<a href="https://colab.research.google.com/github/pierfrancescomartinello/NLP-Project/blob/main/rag_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG pipelines applied to Unipa's website


A HuggingFace API key is necessary:
1. generate it at https://huggingface.co/settings/tokens
2. on the notebook, click on the key icon in the left sidebar
3. click "Add new secret" and name it HF_TOKEN, then insert the generated key


<font color="red">**WARNING:**</font> If on Google Colab, a session restart (e.g. Runtime -> Restart session and run all) is **necessary** after installing dependencies, in order to set all the new installed packages correctly. Also it is suggested to do a single run of the entire code. The last part `Interactive Testing` is dedicated to the interaction with the models.


## Dependencies
- `haystack-ai` is the preview of Haystack 2.0
- `sentence_transformers` is needed for embeddings
- `transformers` is needed to use open-source LLMs
- `accelerate` and `bitsandbytes` are required to use quantized versions of these models (with smaller memory footprint)

In [1]:
%%capture
! pip install haystack-ai transformers accelerate bitsandbytes sentence_transformers

## Setup

In [2]:
from IPython.display import Image
from pprint import pprint
import torch
import rich
import random

In [3]:
!wget https://raw.githubusercontent.com/pierfrancescomartinello/NLP-Project/main/output/unipa_dataset.json

--2024-06-19 13:18:51--  https://raw.githubusercontent.com/pierfrancescomartinello/NLP-Project/main/output/unipa_dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7606379 (7.3M) [text/plain]
Saving to: ‘unipa_dataset.json.2’

unipa_dataset.json. 100%[===================>]   7.25M  --.-KB/s    in 0.06s   

2024-06-19 13:18:51 (125 MB/s) - ‘unipa_dataset.json.2’ saved [7606379/7606379]



In [4]:
import pandas as pd
import numpy as np

# The dataset is loaded from Google Colab
filepath = "/content/unipa_dataset.json"
df = pd.read_json(filepath)
df.columns = ["title", "addr", "text"]

df

,title,addr,text
0,Università degli Studi di Palermo,https://www.unipa.it/,
1,Organi di Governo | Università degli Studi di ...,https://www.unipa.it/ateneo/OrganiDiGovernoECo...,
2,Fatturazione elettronica | Università degli St...,https://www.unipa.it/target/imprese/informazio...,Il D.M 55 del 3 aprile 2013 prevede l'obbligo ...
3,Presentazione | Università degli Studi di Palermo,https://www.unipa.it/ateneo/presentazione/,
4,Credits | Università degli Studi di Palermo,https://www.unipa.it/credits.html,I contenuti della home page e delle relative s...
...,...,...,...
6333,Calendari didattici DARCH | Centro per l’innov...,https://www.unipa.it/strutture/cimdu/Calendari...,Calendario didattico DARCH A.A. 2024/2025 Ca...
6334,| Università degli Studi di Palermo,https://www.unipa.it/amministrazione/rettorato...,
6335,Settore Comunicazione e URP | Settore Comunica...,https://www.unipa.it/amministrazione/rettorato...,
6336,Regolamenti per aree tematiche di interesse | ...,https://www.unipa.it/servizi/prevenzionedellac...,REGOLAMENTI PERSONALE DOCENTE E RICERCATORE


## Preprocessing

In [5]:
import re

# It has been noticed that there are some entries that consist only of a date in a particular format
def remove_dates(s: str) -> str:
    regex = r"\d{1,2}-(gen|feb|mar|apr|mag|giu|lug|ago|set|ott|nov|dic)-\d{4}"

    return re.sub(regex, "", s)

# Empty strings are replaced with the numpy NaN representation so that dropna can be used
def remove_empty_docs(df: str) -> str:
    df["text"] = df["text"].replace("", np.nan)
    df.dropna(subset=["text"], inplace=True)

    return df


def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    df["text"] = df["text"].apply(remove_dates)
    df["text"] = df["text"].apply(lambda x: x.strip())

    df = remove_empty_docs(df)


    # Wpecify minimum document length
    # Most documents below threshold tend to be garbage documents
    df = df[df["text"].str.len() >= 200]

    df.reset_index(inplace=True, drop=True)

    return df

In [6]:
df = preprocess(df)
df

,title,addr,text
0,Fatturazione elettronica | Università degli St...,https://www.unipa.it/target/imprese/informazio...,Il D.M 55 del 3 aprile 2013 prevede l'obbligo ...
1,Credits | Università degli Studi di Palermo,https://www.unipa.it/credits.html,I contenuti della home page e delle relative s...
2,Sostegno allo studio | Centro per l’innovazion...,https://www.unipa.it/strutture/cimdu/Sostegno-...,"Da 15 anni ItaStra, Scuola di Lingua Italiana ..."
3,PNRR | PNRR | Università degli Studi di Palermo,https://www.unipa.it/progetti/pnrr/,Il Piano Nazionale di Ripresa e Resilienza - P...
4,Corsi di preparazione alle prove di accesso A....,https://www.unipa.it/Corsi-di-preparazione-all...,Sono aperte le iscrizioni all’edizione inverna...
...,...,...,...
2771,U.O. Didattica e Internazionalizzazione - Desc...,https://www.unipa.it/strutture/cimdu/U.O.-Dida...,Il Responsabile dell’U.O. Didattica e Internaz...
2772,Ricercatori neoassunti | Università degli Stud...,https://www.unipa.it/strutture/cimdu/ricercato...,Per restare sempre aggiornati è possiblie iscr...
2773,Declaratorie della U.O. | Centro per l’innovaz...,https://www.unipa.it/strutture/cimdu/Declarato...,La U.O. Didattica e Internazionalizzazione si ...
2774,Calendari didattici DARCH | Centro per l’innov...,https://www.unipa.it/strutture/cimdu/Calendari...,Calendario didattico DARCH A.A. 2024/2025 Cal...


In [7]:
from haystack.dataclasses import Document

# Population of the Document dataclasses. Title and URLs are added as metadata
titles = list(df["title"].values)
texts = list(df["text"].values)
urls = list(df["addr"].values)

raw_docs = []
for title, text, url in zip(titles, texts, urls):
    raw_docs.append(Document(content=text, meta={"name": title or "", "url": url}))

## Indexing Pipeline

In [8]:
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.embedders import (
    SentenceTransformersTextEmbedder,
    SentenceTransformersDocumentEmbedder,
)
from haystack.components.generators import HuggingFaceLocalGenerator, HuggingFaceAPIGenerator
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack.components.builders import PromptBuilder
from haystack.utils import ComponentDevice, Secret

In [9]:
# We use In memory Document Store, albeit a different kind, like FAISSDocumentStore should be used for heavier document stores
document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")

In [10]:
# Creation of the indexing pipeline

indexing = Pipeline() # Empty Pipeline
# Cleaning the document from regex, headers, footers and similar
indexing.add_component("cleaner", DocumentCleaner())

# We split the documents in two sentence long subdocument
indexing.add_component(
    "splitter", DocumentSplitter(split_by="sentence", split_length=2)
)

# We transform the sentences in embeddings
indexing.add_component(
    "doc_embedder",
    SentenceTransformersDocumentEmbedder(
        model="thenlper/gte-large", # This is the model used
        device=ComponentDevice.from_str("cuda:0"), # We use the GPU for this operation
        meta_fields_to_embed=["title"], # We add the
    ),
)

# This simply writes in the document store
indexing.add_component(
    "writer",
    DocumentWriter(document_store=document_store, policy=DuplicatePolicy.OVERWRITE),
)

# Cleaner -> Splitter -> Document Embedder -> Writer
indexing.connect("cleaner", "splitter")
indexing.connect("splitter", "doc_embedder")
indexing.connect("doc_embedder", "writer")

🚅 Components
  - cleaner: DocumentCleaner
  - splitter: DocumentSplitter
  - doc_embedder: SentenceTransformersDocumentEmbedder
  - writer: DocumentWriter
🛤️ Connections
  - cleaner.documents -> splitter.documents (List[Document])
  - splitter.documents -> doc_embedder.documents (List[Document])
  - doc_embedder.documents -> writer.documents (List[Document])

In [11]:
# This command takes the raw documents and transform them in embeddings
indexing.run({"cleaner": {"documents": raw_docs}})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Batches:   0%|          | 0/948 [00:00<?, ?it/s]

{'writer': {'documents_written': 30314}}

### Information about the embeddings

Let's inspect the total number of chunked Documents and examine a Document

In [12]:
print(f"We have a total of {len(document_store.filter_documents())} documents in the document store")
print(f"The size of the embeddings is: {len(document_store.filter_documents()[0].embedding)}")

We have a total of 27842 documents in the document store
The size of the embeddings is: 1024


In [13]:
pprint(document_store.filter_documents()[0])

Document(id=c81160b1c9bd873b1ae604c85641fd6e8290df7a920af385697caf26e8afec58, content: 'Il D.M 55 del 3 aprile 2013 prevede l'obbligo della fatturazione elettronica in tutti i rapporti con...', meta: {'name': 'Fatturazione elettronica | Università degli Studi di Palermo', 'url': 'https://www.unipa.it/target/imprese/informazioni/fatturazione-elettronica/', 'source_id': '78eb0a1349eff5f5fda46f5d6efeeffc87096deee81660c984d1f79334123146', 'page_number': 1}, embedding: vector of size 1024)


## RAG Pipeline

It is suggested to run this code only once, because it would create problems with the pipelines. Each element (generator, prompt_builder, ect...) are bounded to only one pipeline, hence if several execution of the same code are done, it will be necessary to change the name of the variables used in pipeline.

In [15]:
# Due to the limited resources, a 4-bit quantization is done
zephyr_generator = HuggingFaceLocalGenerator(
    "HuggingFaceH4/zephyr-7b-beta", #Name of the model
    huggingface_pipeline_kwargs={
        "device_map": "auto",
        "model_kwargs": {
            "load_in_4bit": True,
            "bnb_4bit_use_double_quant": True,
            "bnb_4bit_quant_type": "nf4",
            "bnb_4bit_compute_dtype": torch.bfloat16,
        },
    },
    generation_kwargs={"max_new_tokens": 500},
)

In [16]:
# A warm up is in order, since the generator is saved into memory
zephyr_generator.warm_up()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

### `Llama-3-8B`

In [17]:
# Due to the limited resources, a 4-bit quantization is done
# Warm-up is not possible here since we are loading from remote
from google.colab import userdata

llama_generator = HuggingFaceAPIGenerator(api_type="serverless_inference_api",
                                    api_params={"model": "meta-llama/Meta-Llama-3-8B-Instruct"},
                                    token = Secret.from_token(userdata.get("HF_TOKEN")))

In [18]:
zephyr_prompt_template = """<|system|>Using the information contained in the context, give a comprehensive answer to the question.
If the answer is contained in the context, also report the source URL.
If the answer cannot be deduced from the context, do not give an answer.</s>
<|user|>
Context:
  {% for doc in documents %}
  {{ doc.content }} URL:{{ doc.meta['url'] }}
  {% endfor %};
  Question: {{query}}
  </s>
<|assistant|>
"""
# Since an PromptBuilder object can be associated with only one Pipeline, two copies of the same object had to be made
zephyr_prompt_builder = PromptBuilder(template=zephyr_prompt_template)


llama_prompt_template = """
    Question : {{query}}
    Context:
    {% for doc in documents %}
    {{ doc.content }} URL:{{ doc.meta['url'] }}
    {% endfor %};

"""
llama_prompt_builder = PromptBuilder(template=llama_prompt_template)

### Pipeline Creation

In [19]:
zephyr_rag = Pipeline() # Empty Pipeline
# Text embedder
zephyr_rag.add_component(
    "text_embedder",
    SentenceTransformersTextEmbedder(
        model="thenlper/gte-large", # This is the model for the sentence transformer
        device=ComponentDevice.from_str("cuda:0") # We use the GPU
    ),
)
# Embedding Retriever
zephyr_rag.add_component(
    "retriever", InMemoryEmbeddingRetriever(
        document_store=document_store,
        top_k=5 #Find the 5 most similar documents to the query
    )
)
zephyr_rag.add_component("prompt_builder", zephyr_prompt_builder)
zephyr_rag.add_component("llm", zephyr_generator)

# Text Embedder -> Retriever -> Prompt Builder -> LLM
zephyr_rag.connect("text_embedder", "retriever")
zephyr_rag.connect("retriever.documents", "prompt_builder.documents")
zephyr_rag.connect("prompt_builder.prompt", "llm.prompt")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: HuggingFaceLocalGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [20]:
llama_rag = Pipeline() # Empty Pipeline
# Text embedder
llama_rag.add_component(
    "text_embedder",
    SentenceTransformersTextEmbedder(
        model="thenlper/gte-large",# This is the model for the sentence transformer
        device=ComponentDevice.from_str("cuda:0")# We use the GPU
    ),
)
# Embedding Retriever
llama_rag.add_component(
    "retriever", InMemoryEmbeddingRetriever(
        document_store=document_store,
        top_k=5 #Find the 5 most similar documents to the query
    )
)
llama_rag.add_component("prompt_builder", llama_prompt_builder)
llama_rag.add_component("llm", llama_generator)

# Text Embedder -> Retriever -> Prompt Builder -> LLM
llama_rag.connect("text_embedder", "retriever")
llama_rag.connect("retriever.documents", "prompt_builder.documents")
llama_rag.connect("prompt_builder.prompt", "llm.prompt")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: HuggingFaceAPIGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

## Questions

In [21]:
def get_generative_answer(query, model_object):
    # We pass the query to the particular model
    results = model_object.run(
        {"text_embedder": {"text": query}, "prompt_builder": {"query": query}}
    )
    # A reply is outputted
    answer = results["llm"]["replies"][0]
    rich.print(answer)

In [22]:
def query_both_models(query: str):
    print("ZEPHYR:")
    get_generative_answer(query, zephyr_rag)
    print("\n LLAMA:")
    get_generative_answer(query, llama_rag)

## Test questions

In [23]:
query_both_models("Tell me about Data, Algorithms and Machine Intelligence")

ZEPHYR:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The Master's Degree program in Data, Algorithms, and Machine Intelligence, as described in the context, provides 
students with theoretical, experimental, and practical skills in data representation and computation for efficient 
and intelligent information processing. The program covers transversal topics in Data, Algorithms, and Machine 
Intelligence, as well as in-depth studies in application areas where data are big, relations are complex, and 
machine-learning structures are deep. The program requires either an Italian Bachelor (Laurea) in specific fields 
or a foreign degree equivalent to those listed, as well as English proficiency at the B2 level of the Common 
European Framework of Reference for Languages. Some specific courses offered as part of this program include 
Artificial Intelligence and Deep Learning, Big Data & Analytics, Game Theory, Financial Planning and Control, and 
Categorical Data. This program is part of the LM 18 study program, and more information can be found on the 
websites provided. Additionally, there is an upcoming seminar called "AI 4 FUTURE" that will provide insights into 
harnessing the power of data and understanding its impact on innovation.


 LLAMA:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Answer:
    The Master's Degree program in Data, Algorithms, and Machine Intelligence at the University of Palermo provides
a comprehensive education in data representation and computation, focusing on efficient and intelligent information
processing. The program covers a range of topics, including:

    Data:
        - Data representation and computation
        - Big data management
        - Multisensory data exploration
        - Knowledge representation and reasoning

    Algorithms:
        - Combinatorial and probabilistic algorithms
        - Big data algorithms
        - Pattern discovery for life sciences

    Machine Intelligence:
        - Artificial intelligence and deep learning
        - Machine intelligence for optimization
        - Information retrieval and natural language processing

    The program also covers transversal topics, such as cybersecurity, information theory and data compression, 
data encryption and codes, cloud and high-performance computing, and complex networks.

    The program is designed for students with a background in computer science, mathematics, or physics, and 
requires a strong foundation in mathematics and physics, as well as computer and information sciences. The program 
is taught in English, and students must have a B2 level of English language proficiency.

    The program offers a range of courses, including:

    - Combinatorial and probabilistic algorithms
    - Big data management
    - Multisensory data exploration
    - Knowledge representation and reasoning
    - Cybersecurity
    - Information theory and data compression
    - Data encryption and codes
    - Cloud and high-performance computing
    - Complex networks
    - Artificial intelligence and deep learning
    - Machine intelligence for optimization
    - Information retrieval and natural language processing
    - Pattern discovery for life sciences

    The program is designed to provide students with a strong foundation in data, algorithms, and machine 
intelligence, and to prepare them for careers in a range of fields, including data science, artificial 
intelligence, and machine learning. 
URL:https://www.unipa.it/dipartimenti/matematicaeinformatica/cds/dataalgorithmsandmachineintelligence2270/ 
URL:https://www.unipa.it/dipartimenti/seas/en/educational-offer/ 
URL:https://www.unipa.it/mobilita/docenti-unipa/accordi-di-cooperazione/doppio-titolo-di-laurea/ 
URL:https://www.unipa.it/dipartimenti/ingegneria/cds/ingegneriagestionale2034/Seminar-AI-4-FUTURE-00002/ 
URL:https://www

In [24]:
query_both_models("Dimmi gli eventi dell'Università di Palermo del 2024")

ZEPHYR:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Già dal 11 giugno 2024, alle ore 9.00, si svolgeranno attività presso l'Università degli Studi di Palermo, edificio
3, Viale delle Scienze. Il Rettore dell'Ateneo, prof. Massimo Midiri, ha presentato le attività progettuali 
dell'Università nel contesto della Programmazione per il 2024. Tali attività si riferiscono alla Gruppo di Lavoro 
per il Completamento delle Procedure Concorsuali per la Scuola di Medicina e Chirurgia, come si può leggere dal 
sito web dell'Università di Palermo, disponibile all'indirizzo: 
https://www.unipa.it/servizi/segreterie/Gruppo-di-lavoro-per-il-completamento-delle-procedure-concorsuali--Medicina
-e-Chirurgia/

Inoltre, il 11 giugno 2024, alle ore 9.00, si terrà una cerimonia di inaugurazione dell'Anno Accademico 2024-2025 
all'Aula Magna del Dipartimento di Ingegneria dell'Università di Palermo, Campus di Viale delle Scienze. La 
cerimonia sarà presenziata dalla Presidente del Parlamento europeo Roberta Metsola, come si può leggere dal sito 
web dell'Università di Palermo, disponibile all'indirizzo: 
https://www.unipa.it/Inaugurazione-Anno-Accademico-2023-2024/

Infine, il 22 febbraio 2020, alle ore 11.15, è stata trasmessa in streaming l'Inaugurazione dell'Anno Accademico 
2020-2021, la 215ª dalla fondazione, come si può leggere dal sito web dell'Università di Palermo, disponibile 
all'indirizzo: https://www.unipa.


 LLAMA:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Answer:
    Gli eventi dell'Università di Palermo del 2024 sono:
    - Martedì 11 giugno 2024, dalle ore 9.00, presso l'Università degli Studi di Palermo edificio 3, Viale delle 
Scienze, Palermo, attività progettuali dell'Ateneo nell'ambito della Programmazione per il 2024. 
URL:https://www.unipa.it/target/futuristudenti/Gruppo-di-lavoro-per-il-completamento-delle-procedure-concorsuali--M
edicina-e-Chirurgia/
    - Martedì 11 giugno 2024, dalle ore 9.00, presso l'Università degli Studi di Palermo edificio 3, Viale delle 
Scienze, Palermo, attività progettuali dell'Ateneo nell'ambito della Programmazione per il 2024. 
URL:https://www.unipa.it/servizi/segreterie/Gruppo-di-lavoro-per-il-completamento-delle-procedure-concorsuali--Medi
cina-e-Chirurgia/
    Non ci sono altri eventi specificati per il 2024. Gli eventi del 2020 e del 2023 sono stati già celebrati. 
URL:https://www.unipa.it/?lista=archivio&id=4eb4b438-d856-11e4-9500-005056010139 e 
URL:https://www.unipa.it/Inaugurazione-Anno-Accademico-2023-2024/ rispettivamente. 
URL:https://www.unipa.it?lista=archivio&id=a9c0e626-855f-11ea-9011-005056010139 è un link a una presentazione del 
Rettore dell'Università degli Studi di Palermo prof. Massimo Midiri. 
URL:https://www.unipa.it/?lista=archivio&id=4eb4b438-d856-11e4-9500-005056010139 è un link a un evento del 2020. 
URL:https://www.unipa.it/Inaugurazione-Anno-Accademico-2023-2024/ è un link a un evento del 2023. 
URL:https://www.unipa.it/target/futuristudenti/

In [25]:
query_both_models("Parlami di Giorgio Ausiello")

ZEPHYR:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The question asks for information about Giorgio Ausiello, and we can find a lecture by him titled "Algoritmi nel 
Medioevo: da al-Khwarizmi a Luca Pacioli" that will take place on September 30 at the Sala Magna del Complesso 
Monumentale dello Steri. The source URL is provided. No further information about Ausiello is given in the context.
If you need more details about him, you may need to search for additional sources or contact the organizers of the 
lecture.


 LLAMA:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Answer:
    Il prof. Giorgio Ausiello è un docente del Dipartimento di Scienze e Tecnologie Biomediche e Fisiche (STEBICEF)
dell'Università degli Studi di Palermo. Ha tenuto una Lectio Magistralis dal titolo "Algoritmi nel Medioevo: da 
al-Khwarizmi a Luca Pacioli" presso la Sala Magna del Complesso Monumentale dello Steri. La sua ricerca si 
concentra sugli algoritmi e la loro applicazione nel Medioevo. È anche membro della Commissione di valutazione per 
la Laurea Magistrale in Scienze della Natura. È inoltre coinvolto in vari eventi e seminari presso l'Università 
degli Studi di Palermo. È un docente di rilievo nel campo delle scienze e tecnologie biomediche e fisiche. 
URL:https://www.unipa.it/dipartimenti/stebicef/struttura/regolamento.html
    
    Note:
    - La Lectio Magistralis è un evento accademico che si tiene presso l'Università degli Studi di Palermo.
    - Il Dipartimento di Scienze e Tecnologie Biomediche e Fisiche (STEBICEF) è un dipartimento dell'Università 
degli Studi di Palermo.
    - La Commissione di valutazione per la Laurea Magistrale in Scienze della Natura è un organismo che si occupa 
di valutare le tesi di laurea magistrale in questo campo.
    - L'Università degli Studi di Palermo è un'istituzione di istruzione superiore italiana. 
URL:https://www.unipa.it/dipartimenti/distem/cds/scienzedellanatura2056/Date-colloqui-verifica-requisiti-Laurea-Mag
istrale-00001/ URL:https://www.unipa.it/dipartimenti/scienzeumanistiche/didattica/orientamento/eventi.html 
URL:https://www.unipa.it/dipartimenti/di.gi./Ciclo-di-seminari-propedeutici-alla-partecipazione-alle-attivit-della-
Clinica-legale-migrazioni-e-diritti---ottobre-novembre-202

In [26]:
query_both_models("What were the deans of University of Palermo?")

ZEPHYR:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The provided context lists a comprehensive list of the rectors, or university presidents, of the University of 
Palermo for various historical periods. The list includes:

- Regno di Sicilia:
  - Gabriele Castelli (1781-1800)
  - Gregorio Speciale (1801-1805)
  - Raimondo Palermo (1806-1816)

- Regno delle Due Sicilie:
  - Raimondo Palermo (1816-1840)

- Regno d'Italia:
  - Alessandro Casano (1841-1843)
  - Giuseppe D'Agostino (1844-1855)
  - Giovanni Laviosa (1856-1858)
  - Giovanni Cumbo (1858-1860)
  - Filippo Casoria (1860-1861)
  - Salvatore Cacopardo (1861-1862)
  - Nicolò Musumeci (1863-1865)
  - Stanislao Cannizzaro (1866-1868)
  - Giuseppe Albeggiani (1869-1874)
  - Gaetano Giorgio Gemmellaro (1874-1876)
  - Antonino Garajo (1877-1880)
  - Gaetano Giorgio Gemmellaro (1881-1883)
  - Simone Corleo (1883-1885)
  - Emanuele Paternò (1886-1890)
  - Damiano Macaluso (1891-1893)
  - Giuseppe Gugino (1893-1895)
  - Giuseppe Ricca Salerno (1895-1896)
  - Arturo Marcacci (1896-1898)
  - Gi


 LLAMA:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Answer: The deans of University of Palermo were:
    1. Gabriele Castelli (1781-1800)
    2. Gregorio Speciale (1801-1805)
    3. Raimondo Palermo (1806-1816)
    4. Alessandro Casano (1841-1843)
    5. Giuseppe D'Agostino (1844-1855)
    6. Giovanni Laviosa (1856-1858)
    7. Giovanni Cumbo (1858-1860)
    8. Filippo Casoria (1860-1861)
    9. Salvatore Cacopardo (1861-1862)
    10. Nicolò Musumeci (1863-1865)
    11. Stanislao Cannizzaro (1866-1868)
    12. Giuseppe Albeggiani (1869-1874)
    13. Gaetano Giorgio Gemmellaro (1874-1876)
    14. Antonino Garajo (1877-1880)
    15. Gaetano Giorgio Gemmellaro (1881-1883)
    16. Simone Corleo (1883-1885)
    17. Emanuele Paternò (1886-1890)
    18. Damiano Macaluso (1891-1893)
    19. Giuseppe Gugino (1893-1895)
    20. Giuseppe Ricca Salerno (1895-1896)
    21. Arturo Marcacci (1896-1898)
    22. Giuseppe Gugino (1898-1899)
    23. Adolfo Venturi (1900-1903)
    24. Antonino Salinas (1903-1904)
    25. Luigi Manfredi (1904-1908)
    26. Salvatore Riccobono (1908-1911)
    27. Raffaele Federico (1911-1914)
    28. Gaetano Mario Columba (1914-1918)
    29. Francesco Spallitta (1918-1921)
    30. Salvatore Di Marzo (1921-1923)
    31. Francesco Ercole (1924-1932)

In [27]:
query_both_models("Quali materie sono insegnate alla Laurea Triennale in Informatica dell'Università di Palermo?")

ZEPHYR:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

La question è contenuta nel contesto.

Le materie insegnate alla Laurea Triennale in Informatica dell'Università di Palermo sono trattate nel Manifesto 
degli studi del corso di laurea triennale in Informatica, che è disponibile all'indirizzo 
https://www.unipa.it/dipartimenti/matematicaeinformatica/cds/informatica2086/index.html?lista=archivio&id=7b18ac33-
fb65-11e3-998d-005056010139.

Il corso di laurea triennale in Informatica dell'Università di Palermo, attivo dal 2014/15, tratta tematiche 
fondamentali dell'Informatica come la Teoria dell’Informazione e i modelli di calcolo, e approfondimenti in settori
specifici dedicati alla progettazione di algoritmi avanzati, ai metodi per la gestione e l'analisi di grandi 
quantità di dati, alle tecniche per la progettazione di infrastrutture e reti ad alte prestazioni, con particolare 
riferimento alla qualità del servizio, la sicurezza e la protezione per la privacy dei dati, allo sviluppo web e 
alle tecnologie mobili.

La Laurea Magistrale in Informatica, attiva dal 2014/15 e unica laurea magistrale nell'Ateneo di Palermo ad aver 
ottenuto per il 2015 la certificazione di qualità del GRIN, tratta queste stesse materie con un approfondimento più
avanzato.


 LLAMA:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Answer: 
    La Laurea Triennale in Informatica dell'Università di Palermo non è menzionata nel testo. Il testo parla di 
Laurea Magistrale in Informatica e di Laurea Magistrale in Ingegneria Informatica. Non è disponibile informazione 
sulla Laurea Triennale in Informatica dell'Università di Palermo. 
    However, according to the Manifesto degli studi del corso di laurea triennale in Informatica A., the course 
includes the following subjects:
    - Fondamenti di Informatica
    - Programmazione
    - Strutture di Dati
    - Sistemi Operativi
    - Reti di Calcolatori
    - Informatica e Società
    - Progettazione di Sistemi Informatici
    - Laboratorio di Informatica
    - Tirocinio
    - Tirocinio esterno

    Please note that the information provided is based on the Manifesto degli studi del corso di laurea triennale 
in Informatica A. and may be subject to change. It is recommended to check with the University of Palermo for the 
most up-to-date information on the course.

In [28]:
query_both_models("Dimmi gli orari della Biblioteca del Dipartimento di Matematica e Informatica dell'Università di Palermo")

ZEPHYR:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

La biblioteca del Dipartimento di Matematica e Informatica dell'Università di Palermo chiude eccezionalmente alle 
ore 13.30 il martedì 28 maggio, come si comunica nel link: 
https://www.unipa.it?lista=archivio&id=40857010-56a5-11e9-ae2b-005056010139.


 LLAMA:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Answer:
    Gli orari della Biblioteca del Dipartimento di Matematica e Informatica dell'Università di Palermo sono:
    - Apertura: non specificati
    - Chiusura: martedì 28 maggio alle ore 13:30 (eccezionale) - altrimenti non specificati
    
    Nota: gli orari della biblioteca non sono stati specificati, quindi non è possibile fornire informazioni più 
precise. È stato notato solo che la biblioteca chiuderà eccezionalmente alle ore 13:30 il martedì 28 maggio. È 
consigliabile contattare direttamente il Dipartimento di Matematica e Informatica o la biblioteca per ottenere 
informazioni più precise sugli orari di apertura e chiusura.

In [29]:
query_both_models("Dimmi gli eventi e i seminari del Dipartimento di Matematica e Informatica dell'Università di Palermo del 2023")

ZEPHYR:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Based on the context provided, there are two events mentioned for the Department of Mathematics and Computer 
Science at the University of Palermo in 2023. 

1. On November 9, 2023, there will be an online event. The exact nature of the event is not specified, but it can 
be found on the department's website at the following URL: 
https://www.unipa.it/dipartimenti/scienzeumanistiche/Terzamissione/Eventi_pubblici/

2. There is also a seminar scheduled for October 18, 2023, which will be held online. Again, the specific details 
of the seminar are not provided, but it can be found on the department's website at the following URL: 
https://www.unipa.it/dipartimenti/matematicaeinformatica/index.html?lista=archivio&id=b604d3f5-2e5a-11e2-b246-11995
6010099

Unfortunately, there is no information provided about any other events or seminars for the Department of 
Mathematics and Computer Science at the University of Palermo in 2023.


 LLAMA:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Answer:
    The events and seminars of the Department of Mathematics and Computer Science of the University of Palermo in 
2023 are:
    1. Seminar by Dr. Beatrice Donelli, Department of Mathematics and Computer Science Freie Universität Berlin, 
Germany, on May 8, 2024, at 14:00 in room F, DiFC, Via Archirafi 36, Palermo.
    2. Open Day of the Department of Mathematics and Computer Science, on , at , at the Department's location in 
Via Archirafi 34.
    3. Public event on November 9, 2023, at 12, Viale delle Scienze, Palermo.
    4. Online event on October 18, 2023.
    5. Convention between the Department of Mathematics and Computer Science and the Circolo Matematico di Palermo,
signed on January 17, 2014.
    Note: The events and seminars listed are based on the provided text and may not be exhaustive or up-to-date. It
is recommended to check with the Department of Mathematics and Computer Science or the University of Palermo for 
the most recent and accurate information.

In [30]:
query_both_models("Quali professori dell'Università di Palermo che insegnano materie che contengono la parola 'Programmazione'?")

ZEPHYR:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The context provided does not include information about specific professors who teach courses related to 
programming. Therefore, it is not possible to provide a comprehensive answer to this question based on the given 
context. If the university's website or other official sources provide a list of professors and their courses, it 
may be possible to determine which professors teach programming-related courses. However, without this information,
it is not possible to provide a definitive answer.


 LLAMA:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Answer:
    I professori dell'Università di Palermo che insegnano materie che contengono la parola 'Programmazione' sono:
    - Prof. Francesco Amato (Dipartimento di Ingegneria Informatica e delle Telecomunicazioni)
    - Prof. Giovanni Denaro (Dipartimento di Ingegneria Informatica e delle Telecomunicazioni)
    - Prof. Giuseppe M. Plano (Dipartimento di Ingegneria Informatica e delle Telecomunicazioni)
    - Prof. Salvatore Monteleone (Dipartimento di Ingegneria Informatica e delle Telecomunicazioni)
    - Prof. Vincenzo Nicosia (Dipartimento di Ingegneria Informatica e delle Telecomunicazioni)
    - Prof. Giovanni Russo (Dipartimento di Ingegneria Informatica e delle Telecomunicazioni)
    - Prof. Salvatore Ruggiero (Dipartimento di Ingegneria Informatica e delle Telecomunicazioni)
    - Prof. Giovanni Schembra (Dipartimento di Ingegneria Informatica e delle Telecomunicazioni)
    - Prof. Salvatore Vitabile (Dipartimento di Ingegneria Informatica e delle Telecomunicazioni)
    - Prof. Giovanni Zappalà (Dipartimento di Ingegneria Informatica e delle Telecomunicazioni)
    - Prof. Salvatore Zappalà (Dipartimento di Ingegneria Informatica e delle Telecomunicazioni)
    - Prof. Giovanni Zappalà (Dipartimento di Ingegneria Informatica e delle Telecomunicazioni)
    - Prof. Salvatore Zappalà (Dipartimento di Ingegneria Informatica e delle Telecomunicazioni)
    - Prof. Giovanni Zappalà (Dipartimento di Ingegneria Informatica e delle Telecomunicazioni)
    - Prof. Salvatore Zappalà (Dipartimento di Ingegneria Informatica e delle Telecomunicazioni)
    - Prof. Giovanni Zappalà (Dipartimento di Ingegneria Informatica e delle Telecomunicazioni)
    - Prof. Salvatore Zappalà (Dipartimento di Ingegneria Informatica e delle Telecomunicazioni)
    - Prof.

In [31]:
query_both_models("Explain the history of the University of Palermo")

ZEPHYR:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The University of Palermo, located in the city of Palermo in Sicily, has a rich history that has been influenced by
the location where it is situated. While a more detailed account can be found on the university's website 
(https://www.unipa.it/note-legali-e-privacy/en/note-legali/index.html), here is a brief overview:

The university's origins can be traced back to the 14th century, when King Peter I of Aragon founded a Studium 
Generale in Palermo. This institution provided education in law, medicine, and theology, and was one of the first 
universities in Europe. However, it was closed down by the Spanish Inquisition in the 16th century.

The university was reestablished in 1806 by Napoleon Bonaparte, who recognized the importance of education in 
promoting economic and social development. The new university offered courses in law, medicine, and philosophy, and
was open to both men and women.

During the 19th and 20th centuries, the university faced various challenges, including political instability and 
economic hardship. However, it continued to evolve and expand, adding new faculties and departments to keep up with
the changing needs of society.

Today, the University of Palermo is a modern institution with 16 departments covering a wide range of disciplines, 
from engineering and computer science to humanities and social sciences. It is committed to providing high-quality 
education, conducting cutting-edge research, and contributing to the social and economic development of Sicily and 
beyond.

Sources:
- "History" section on the university's website 
(https://www.unipa.it/mobilita/en/useful-information/about-palermo-university/)
- "Legal and Privacy Notice" section on the university's website 
(https://www.unipa.it/note-legali-e-privacy/en/note-legali/index.html)
- "Departments" section on the university's website (https://www.unipa.it/dipartimenti/)
- "Teaching and Learning" section on the website of the Department of Law (https://www.unipa.it/dipartiment


 LLAMA:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Answer:
    The University of Palermo has a rich history that dates back to the 12th century. The city of Palermo, where 
the university is located, has been an important cultural and commercial center in the Mediterranean for centuries.
The university has undergone several transformations and reorganizations over the years, reflecting the changing 
needs and priorities of the region.

    The University of Palermo was founded in 1321 by King Frederick II of the Holy Roman Empire, who was also the 
King of Sicily. At that time, the university was known as the Studium Generale and was one of the first 
universities in Europe. The university was initially focused on the study of law, medicine, and theology, and was 
attended by students from all over Europe.

    Over the centuries, the university underwent several transformations and reorganizations. In the 15th century, 
the university was reorganized and renamed the University of Palermo. In the 18th century, the university was 
closed by the Bourbon monarchy, but it was reopened in the 19th century.

    In the 20th century, the university underwent significant changes and expansions. In 1955, the university was 
reorganized and renamed the University of Palermo. In the 1960s and 1970s, the university underwent a period of 
rapid expansion, with the establishment of new departments and faculties.

    Today, the University of Palermo is a major research and teaching institution in Italy, with a strong focus on 
scientific and technological research. The university has 16 departments and offers a wide range of undergraduate 
and graduate programs in fields such as medicine, engineering, law, and the humanities.

    The university is also known for its strong international connections and partnerships, with collaborations 
with universities and research institutions around the world. The university is also committed to promoting 
cultural and scientific exchange between Italy and other countries.

    In summary, the University of Palermo has a rich history that dates back to the 12th century. The university 
has undergone several transformations and reorganizations over the years, reflecting the changing needs and 
priorities of the region. Today, the university is a major research and teaching institution in Italy, with a 
strong focus on scientific and technological research. 
URL:https://www.unipa.it/mobilita/en/useful-information/about-palermo-university/
    ;
    References:
    1. University of Palermo. (n.d.). About Palermo University. Retrieved from 
<https://www.unipa.it/mobilita/en/useful-information/about

In [32]:
query_both_models("Che cos'è il Palazzo Steri?")

ZEPHYR:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Il Palazzo Steri è una struttura situata in Piazza Marina, 61, che oggi ospita la Sala Carapezza del Rettorato 
dell'Università degli Studi di Palermo (URL: https://www.unipa.it/strutture/scuolaitalianastranieri/). Durante la 
visita guidata a cura del servizio guide, è possibile esplorare i tesori dello Steri (URL: 
https://www.unipa.it/strutture/scuolaitalianastranieri/). La conoscenza della storia secolare del palazzo è emersa 
in parte grazie alle ricerche del professore Camillo Filangeri, ma la storia successiva non è ancora nota in 
molteplici aspetti (URL: https://www.unipa.it/dipartimenti/architettura/struttura/presentazione.html). Un'altra 
struttura dell'Università degli Studi di Palermo, l'Edificio 8 situato in Viale delle Scienze, ha ospitato il 
Dipartimento di Progetto e Costruzione Edilizia e il Dipartimento di Rappresentazione fino al 31 dicembre 2010 
(URL: https://www.unipa.it/dipartimenti/architettura/struttura/presentazione.html). La realizzazione in pietra a 
vista di questa parte basamentale ancora visibile (estesa per nove campate) testimonia dell'impegno economico e 
della qualità di dettaglio richiesta nella fabbrica (URL: 
https://www.unipa.it/dipartimenti/architettura/struttura/presentazione.html). Il 11 gennaio, dalle 9.00, si tiene 
la Giornata della didattica innovativa nel Complesso Monumentale dello Steri (URL: 
https://www.unipa.it/dipartimenti/bi.n.d./Ciclo-di-seminari-Giornata-della-Didattica-Innovativa/).


 LLAMA:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Answer: Il Palazzo Steri è un complesso monumentale che ospita il Rettorato dell'Università degli Studi di 
Palermo e che è sede di vari dipartimenti e strutture dell'università. È situato in piazza Marina, 61, a Palermo. 
Il palazzo ha una storia secolare e ha ospitato diverse istituzioni e strutture nel corso dei secoli. È noto per la
sua architettura e la sua storia, e ospita anche eventi e manifestazioni culturali e didattiche.  ;  ;  ;  ;  ;  ; 
;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;
;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;
;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;
;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;
;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;  ;

In [33]:
query_both_models("Who is Massimo Midiri?")

ZEPHYR:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Massimo Midiri is the Rettore, or rector, of the University of Studies of Palermo, as indicated in the following 
URLs:

- https://www.unipa.it/strutture/orientamento/eventi/welcome-day-lauree-magistrali-2024/
- https://www.unipa.it/dipartimenti/bi.n.d./Welcome-Day-delle-Lauree-Magistrali-2024--00001/
- https://www.unipa.it/strutture/orientamento/eventi/welcome-day-lauree-magistrali-2024/
- https://www.unipa.it/dipartimenti/bi.n.d./Welcome-Day-Lauree-Magistrali-BiND---2023/

In the last URL, Massimo Midiri is also listed as the rector, but the URL also mentions Prof. Enzo Bivona, who is 
identified as the Vice-Direttore, or vice-dean, of the Department of Political Sciences and International 
Relations. However, this does not change the fact that Massimo Midiri is the rector of the university.

Therefore, based on the provided context, Massimo Midiri is the rector of the University of Studies of Palermo.


 LLAMA:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Answer: Massimo Midiri is the Rettore (Chancellor) of the University of Palermo.

## Interactive Testing

### Zephyr

In [ ]:
raise Exception("Execution has successfully terminated! Feel free to inspect the answers above or manually query the system below.")

In [ ]:
try:
    while (query := input("Input your query (type EXIT to finish): ")) != "EXIT":
        print(query)
        get_generative_answer(query, zephyr_rag)
        print("\n\n")
except KeyboardInterrupt:
    pass

### Falcon

In [ ]:
try:
    while (query := input("Input your query (type EXIT to finish): ")) != "EXIT":
        print(query)
        get_generative_answer(query, llama_rag)
        print("\n\n")
except KeyboardInterrupt:
    pass

### Both

In [ ]:
try:
    while (query := input("Input your query (type EXIT to finish): ")) != "EXIT":
        print(query)
        query_both_models(query)
        print("\n\n")
except KeyboardInterrupt:
    pass

______